In [1]:
import numpy as np

grid = [
    "w www www wwwww www w w w o www 000 000 www w",
    "0 000 000  w www wwwww wwwww www           ",
    "    w    O    o    w                       ",
    "www www 000 000 wwwww w 0 0000000 0 0 w w www 0000000 0 www w",
    "    w    o    000    O    w                ",
    "w wwwww www wwwww www w o    o o    o w    ",
    "w o wwwww wwwww o www w o                  ",
    "w www www www www www w    o    g    o    w"
]

max_length = max(len(row) for row in grid)
grid = [list(row.ljust(max_length)) for row in grid]


rewards = {
    'w': -1000,  # Wall (invalid move)
    'o': -10,    # Obstacle (penalty)
    'g': 100,    # Goal (reward)
    '0': -1      # Free space (small penalty for each step)
}

#GridWorld environment
class GridWorld:
    def __init__(self, grid, rewards):
        self.grid = np.array(grid)
        self.rewards = rewards
        self.start = (0, 0)  # Top-left corner
        self.goal = None
        self.state = self.start
        self.actions = ['up', 'down', 'left', 'right']
        self.done = False

        #goal position
        for i in range(len(self.grid)):
            for j in range(len(self.grid[i])):
                if self.grid[i, j] == 'g':
                    self.goal = (i, j)
                    break
            if self.goal:
                break

    def reset(self):
        self.state = self.start
        self.done = False
        return self.state

    def step(self, action):
        if self.done:
            return self.state, 0, True

        i, j = self.state
        if action == 'up':
            i -= 1
        elif action == 'down':
            i += 1
        elif action == 'left':
            j -= 1
        elif action == 'right':
            j += 1

        # Is new position valid?
        if i < 0 or i >= len(self.grid) or j < 0 or j >= len(self.grid[0]):
            return self.state, self.rewards['w'], True  # Invalid move (wall)

        # Is new position a wall?
        if self.grid[i, j] == 'w':
            return self.state, self.rewards['w'], True  # Wall (invalid move)

        # Update state
        self.state = (i, j)

        # Is goal reached?
        if self.state == self.goal:
            self.done = True
            return self.state, self.rewards['g'], True

        # Return the reward for the new state
        reward = self.rewards.get(self.grid[i, j], -1)
        return self.state, reward, False

# Create the environment
env = GridWorld(grid, rewards)

In [2]:
def value_iteration(env, gamma=0.9, theta=1e-6):
    # Initialize the value function
    V = np.zeros(env.grid.shape)

    while True:
        delta = 0
        for i in range(env.grid.shape[0]):
            for j in range(env.grid.shape[1]):
                if env.grid[i, j] == 'w':  # Skip walls
                    continue

                v = V[i, j]
                max_v = -np.inf

                # Compute the value for each action
                for action in env.actions:
                    env.state = (i, j)
                    next_state, reward, done = env.step(action)
                    next_i, next_j = next_state

                    # Skip invalid states (walls)
                    if reward == env.rewards['w']:
                        continue

                    # Update the value
                    max_v = max(max_v, reward + gamma * V[next_i, next_j])

                # Update the value function
                V[i, j] = max_v

                # Update delta (skip inf values)
                if not np.isinf(v) and not np.isinf(V[i, j]):
                    delta = max(delta, abs(v - V[i, j]))

        if delta < theta:
            break

    # Extract the optimal policy
    policy = np.zeros(env.grid.shape, dtype=object)
    for i in range(env.grid.shape[0]):
        for j in range(env.grid.shape[1]):
            if env.grid[i, j] == 'w':  # Skip walls
                continue

            max_action = None
            max_v = -np.inf

            # Find the best action for each state
            for action in env.actions:
                env.state = (i, j)
                next_state, reward, done = env.step(action)
                next_i, next_j = next_state

                # Skip invalid states (walls)
                if reward == env.rewards['w']:
                    continue

                # Update the best action
                v = reward + gamma * V[next_i, next_j]
                if v > max_v:
                    max_v = v
                    max_action = action

            policy[i, j] = max_action

    return V, policy

# Run Value Iteration
V, policy = value_iteration(env)
print("Value Function:")
print(V)
print("Optimal Policy:")
print(policy)

Value Function:
[[ 0.00000000e+00 -8.98144994e-08  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -8.98144994e-08  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -8.98144994e-08  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00            -inf
   0.00000000e+00  0.00000000e+00  0.00000000e+00            -inf
   0.00000000e+00            -inf  0.00000000e+00            -inf
   0.00000000e+00 -8.98144994e-07 -8.98144994e-08 -9.78978044e-07
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.98144994e-08
  -8.98144994e-08 -8.98144994e-08 -8.98144994e-08 -8.98144994e-08
  -8.98144994e-08 -8.98144994e-08 -8.98144994e-08 -8.98144994e-08
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.98144994e-08
   0.00000000e+00 -8.98144994e-08 -8.98144994e-08 -8.98144994e-08
  -8.98144994e-08 -8.98144994e-08 -8.98144994e-08 -8.98144994e-08
  -8.98144994e-08 -8.98144994e-08 -8.98144994e-08 -8.98144994e-08
  -8.98144994e-08 -8.98144994e-08 -8.98144994e-08 -8.9814499

In [3]:
def q_learning(env, gamma=0.9, alpha=0.1, epsilon=0.1, episodes=1000):
    # Initialize the Q-table
    Q = np.zeros((env.grid.shape[0], env.grid.shape[1], len(env.actions)))

    for episode in range(episodes):
        state = env.reset()
        done = False

        while not done:
            i, j = state

            # Choose an action (epsilon-greedy)
            if np.random.rand() < epsilon:
                action = np.random.choice(env.actions)
            else:
                action = env.actions[np.argmax(Q[i, j])]

            # Take the action
            next_state, reward, done = env.step(action)
            next_i, next_j = next_state

            # Skip invalid states (walls)
            if reward == env.rewards['w']:
                continue

            # Update the Q-value
            Q[i, j, env.actions.index(action)] += alpha * (
                reward + gamma * np.max(Q[next_i, next_j]) - Q[i, j, env.actions.index(action)]
            )

            state = next_state

    # Extract the optimal policy
    policy = np.zeros(env.grid.shape, dtype=object)
    for i in range(env.grid.shape[0]):
        for j in range(env.grid.shape[1]):
            if env.grid[i, j] == 'w':  # Skip walls
                continue
            policy[i, j] = env.actions[np.argmax(Q[i, j])]

    return Q, policy

# Run Q-Learning
Q, policy = q_learning(env)
print("Q-Table:")
print(Q)
print("Optimal Policy:")
print(policy)

Q-Table:
[[[ 0.         -0.94185026  0.         -0.89058101]
  [ 0.         -0.199       0.          0.        ]
  [ 0.          0.          0.          0.        ]
  ...
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 0.          0.          0.         -0.19      ]
  [-0.19       -0.1        -0.1        -0.1       ]
  [ 0.          0.          0.          0.        ]
  ...
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 0.          0.          0.          0.        ]
  [-0.1         0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  ...
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 ...

 [[ 0.          0.       